In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
def jac_to_ic(rg,rp,theta,m1,m2,m3):
    
    thrad=np.radians(theta)

    mred2 = m2/(m2+m3)
    mred3 = m3/(m2+m3)

    r12=np.sqrt(rg*rg + mred3*mred3*rp*rp + 2.0*rg*mred3*rp*np.cos(thrad))
    r13=np.sqrt(rg*rg + mred2*mred2*rp*rp - 2.0*rg*mred2*rp*np.cos(thrad))
    r23=rp

    return r12,r13,r23

def ic_to_jac(r12,r13,r23,m1,m2,m3):

    mred2 = m2/(m1+m2)
    d1 = mred2*r12
    cbeta = (r12*r12+r13*r13-r23*r23)/(2.0*r12*r13)

    rp = r12
    rg = np.sqrt(r13*r13+d1*d1-2.0*r13*d1*cbeta)
    ctheta = (r12*r12+d1*d1-rg*rg)/(2.0*rg*d1)
    theta = np.degrees(np.arccos(ctheta))

    return rg,rp,theta

def transform_coords(rg,rp,theta,m1,m2,m3):

    r12,r13,r23=jac_to_ic(rg,rp,theta,m1,m2,m3)
    rgp,rpp,thetap=ic_to_jac(r12,r13,r23,m1,m2,m3)

    return rgp,rpp,thetap

In [15]:
print(transform_coords(2.65,1.4,90,30.9737620,2.0141017778,2.0141017778))

(np.float64(2.5966922514026716), np.float64(2.7408940147331493), np.float64(23.387767598541146))


In [13]:
# ============================================================
#  COORDINATE TRANSFORMATION MODULE
#  (Jacobi <-> Internal coordinates for triatomic systems)
# ============================================================

def jac_to_ic(rg, rp, theta, m1, m2, m3):
    """
    From Jacobi coordinates (reactive channel) to internal distances.

    Parameters
    ----------
    rg : float
        Jacobi distance between atoms 2 and 3.
    rp : float
        Jacobi distance between atom 1 and COM(2-3).
    theta : float
        Angle between rg and rp (in degrees).
    m1, m2, m3 : float
        Atomic masses of atoms 1, 2, 3.

    Returns
    -------
    r12, r13, r23 : floats
        Internal distances between atoms (1-2, 1-3, 2-3).
    """
    thrad = np.radians(theta)
    a2 = m3 / (m2 + m3)
    a3 = m2 / (m2 + m3)

    r12 = np.sqrt(rp**2 + (a2*rg)**2 + 2.0*a2*rp*rg*np.cos(thrad))
    r13 = np.sqrt(rp**2 + (a3*rg)**2 - 2.0*a3*rp*rg*np.cos(thrad))
    r23 = rg

    return r12, r13, r23


def ic_to_jac(r12, r13, r23, m1, m2, m3):
    """
    From internal coordinates to Jacobi coordinates (product channel).

    Parameters
    ----------
    r12, r13, r23 : float
        Internal distances between atoms (1-2, 1-3, 2-3).
    m1, m2, m3 : float
        Atomic masses of atoms 1, 2, 3.

    Returns
    -------
    rg, rp, theta : floats
        Jacobi coordinates (r_g, r_p, theta in degrees) for the product channel.
    """
    # Mass fractions for diatomic 1–2
    a1 = m2 / (m1 + m2)
    a2 = m1 / (m1 + m2)

    # Angles inside the triangle (cosines)
    cbeta  = (r12**2 + r13**2 - r23**2) / (2.0*r12*r13)
    cbeta  = np.clip(cbeta, -1.0, 1.0)
    calpha = (r12**2 + r23**2 - r13**2) / (2.0*r12*r23)
    calpha = np.clip(calpha, -1.0, 1.0)

    # Center of mass of (1–2)
    d1 = a1 * r12
    d2 = a2 * r12

    # Jacobi coordinates (products)
    rg = r12
    rp = np.sqrt(r13**2 - 2.0*d1*r13*cbeta + d1**2)
    ctheta = (r13**2 - d1**2 - rp**2) / (2.0*d1*rp)
    ctheta = np.clip(ctheta, -1.0, 1.0)
    theta = np.degrees(np.arccos(ctheta))

    return rg, rp, theta


# ============================================================
#  SELF-CONSISTENCY TEST
# ============================================================
if __name__ == "__main__":
    # Example system: P + D2 -> PD + D
    m1, m2, m3 = 30.973620, 2.01401778, 2.01401778
    rg, rp, theta = 2.65, 1.4, 90.0

    print("Initial Jacobi (reactants):")
    print(f"rg={rg:.6f}, rp={rp:.6f}, theta={theta:.3f}°")

    # Forward transform: Jacobi -> Internal
    r12, r13, r23 = jac_to_ic(rg, rp, theta, m1, m2, m3)
    print("\nInternal distances:")
    print(f"r12={r12:.6f}, r13={r13:.6f}, r23={r23:.6f}")

    # Backward transform: Internal -> Jacobi (products)
    rg2, rp2, th2 = ic_to_jac(r12, r13, r23, m1, m2, m3)
    print("\nRecovered Jacobi (products):")
    print(f"rg={rg2:.6f}, rp={rp2:.6f}, theta={th2:.6f}°")

    print("\nDifferences:")
    print(f"Δrg={rg2-rg:+.3e}, Δrp={rp2-rp:+.3e}, Δθ={th2-theta:+.3e}")


Initial Jacobi (reactants):
rg=2.650000, rp=1.400000, theta=90.000°

Internal distances:
r12=1.927596, r13=1.927596, r23=2.650000

Recovered Jacobi (products):
rg=1.927596, rp=1.924713, theta=90.347133°

Differences:
Δrg=-7.224e-01, Δrp=+5.247e-01, Δθ=+3.471e-01
